In [14]:
!pip install ray[tune]
!pip install optuna






  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
   ---------------------------------------- 97.9/97.9 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.2 kB ? eta -:--:--
   ---------------------------------------- 75.2/75.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   -------- ------------------------------- 5.4/25.9 MB 115.7 MB/s eta 0:00:01
   ---------------- ----------------------- 11.0/25.9 MB 108.8 MB/s eta 0:00:01
   ----------------------- ---------------- 15.5/25.9 MB 108.8 MB/s eta 0:00:01
   ----------------------------- ---------- 19.1/25.9 MB 93.9 MB/s eta 0:00:01
   -------------------------------- ------- 20.8/25.9 MB 72.6 MB/s eta 0:00:01
   ---------------------------------- ----- 22.4/25.9 MB 59.5 MB/s eta 0:00:01
   -------------

In [3]:
import sys
print(sys.executable)
print(sys.version)

!pip install numpy
!pip install torch
import numpy
print(numpy.__version__)

!pip install transformers[torch]
!pip install transformers
!pip install scikit-learn
!pip install sentencepiece
!pip install protobuf

!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118




c:\Users\igorm\AppData\Local\Programs\Python\Python311\python.exe
3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]
1.24.0
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [23]:

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU disponivel')
    print('Número de GPUs disponíveis:', torch.cuda.device_count())
    print('Nome da GPU:', torch.cuda.get_device_name(0))

else:
    device = torch.device('cpu')
    print('GPU nao disponivel, utilizando CPU')

GPU disponivel
Número de GPUs disponíveis: 1
Nome da GPU: NVIDIA GeForce RTX 2070


In [24]:
import re
def preprocess_text(text):
    # Remove caracteres especiais e múltiplos espaços
    text = re.sub(r'\s+', ' ', text)  # Remove múltiplos espaços
    text = re.sub(r'[^\w\s,]', '', text)  # Remove caracteres especiais, exceto vírgulas
    
    # Converte para minúsculas
    text = text.lower()
    
    # Remove palavras irrelevantes (exemplo)
    irrelevant_words = ['marca', 'oem', 'descrição', 'características', 'condição do item', 'novo', 'frequentes', 'produto', 'nota fiscal', 
                        'garantia', 'meses', 'suporte', 'horário de atendimento', 'equipes']
    
    for word in irrelevant_words:
        text = re.sub(r'\b' + word + r'\b', '', text)
    
    # Padronização do formato
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [25]:
import json
from sklearn.model_selection import train_test_split

with open('output.json', encoding='utf-8') as f:
    data = json.load(f)

inputs = [item['input'] for item in data]
outputs = [
    f"code: {item['output']['catalog_catalog.code']} description: {item['output']['catalog_catalog.description']} "
    f"full_description: {item['output']['catalog_catalog.full_description']} similar_parts: {item['output']['catalog_catalog.similar_parts']} "
    f"application_table: {item['output']['catalog_catalog.application_table']} brand_producer: {item['output']['catalog_catalog.brand_producer']}"
    for item in data
]

inputs = [preprocess_text(text) for text in inputs]


In [26]:
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(inputs, outputs, test_size=0.2, random_state=42)


In [27]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

model_name = "unicamp-dl/ptt5-base-portuguese-vocab"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

class T5Dataset(Dataset):
    def __init__(self, inputs, outputs, tokenizer, max_length=512):
        self.inputs = inputs
        self.outputs = outputs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]

        # Tokenização
        input_encoding = self.tokenizer(
            input_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )
        output_encoding = self.tokenizer(
            output_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )

        input_ids = input_encoding['input_ids'].squeeze()
        attention_mask = input_encoding['attention_mask']
        labels = output_encoding['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }
    
train_dataset = T5Dataset(train_inputs, train_outputs, tokenizer)
test_dataset = T5Dataset(test_inputs, test_outputs, tokenizer)



train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [28]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import os
import tempfile


temp_dir = tempfile.gettempdir()
results_dir = os.path.join(temp_dir, 'results')
logs_dir = os.path.join(temp_dir, 'logs')


def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


ensure_dir(results_dir)
ensure_dir(logs_dir)

print(f"Results directory exists: {os.path.exists(results_dir)}")
print(f"Logs directory exists: {os.path.exists(logs_dir)}")


training_args = TrainingArguments(
    output_dir=results_dir,
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logs_dir,
    logging_steps=10,
    save_steps=10,
    evaluation_strategy="steps"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


trainer.train()

model_save_path = "./results_model/t5_model"
tokenizer_save_path = "./results_model/t5_tokenizer"


os.makedirs(model_save_path, exist_ok=True)
os.makedirs(tokenizer_save_path, exist_ok=True)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Modelo salvo em {model_save_path}")
print(f"Tokenizador salvo em {tokenizer_save_path}")


Results directory exists: True
Logs directory exists: True


{'loss': 35.7334, 'grad_norm': 85.39697265625, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.24}


100%|██████████| 11/11 [00:12<00:00,  1.26s/it]


{'eval_loss': 49.64768981933594, 'eval_runtime': 13.4877, 'eval_samples_per_second': 3.04, 'eval_steps_per_second': 0.816, 'epoch': 0.24}



  6%|▌         | 10/164 [01:40<22:21,  8.71s/it]


...

{'eval_loss': 1.224413275718689, 'eval_runtime': 28.5927, 'eval_samples_per_second': 1.434, 'eval_steps_per_second': 0.385, 'epoch': 3.9}



 98%|█████████▊| 160/164 [35:45<00:42, 10.62s/it]


{'train_runtime': 2193.3333, 'train_samples_per_second': 0.299, 'train_steps_per_second': 0.075, 'train_loss': 21.632778202615132, 'epoch': 4.0}


100%|██████████| 164/164 [36:33<00:00, 13.37s/it]


Modelo salvo em ./results_model/t5_model
Tokenizador salvo em ./results_model/t5_tokenizer


In [34]:
def generate_output(input_text):
  
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)


    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    
    # movendo de volta para a CPU 
    output_text = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)
    print(output_text)
    
    output_dict = {
        "catalog_catalog.code": "",
        "catalog_catalog.full_description": "N/D",
        "catalog_catalog.description": "",
        "catalog_catalog.similar_parts": "N/D",
        "catalog_catalog.application_table": "",
        "catalog_catalog.brand_producer": "BMW Motorrad"
    }

    # Parse the output_text based on your specific requirements
    output_values = output_text.split(",")
    if len(output_values) >= 1:
        output_dict["catalog_catalog.code"] = output_values[0].strip()
    if len(output_values) >= 2:
        output_dict["catalog_catalog.description"] = output_values[1].strip()
    if len(output_values) >= 3:
        output_dict["catalog_catalog.application_table"] = output_values[2].strip()

    return output_dict

# Certifique-se de que o modelo esteja na GPU
model.to(device)

input_text = "11428409567\tFiltro De Oleo Bmw Motorrad F 800 St 2-cyl 06-12\tMarca: BMW Motorrad, Número de peça: 11428409567, OEM: 11428409567, \tDescriçãoFILTRO DE OLEO Bmw Motorrad F 800 ST 2-Cyl 06/12Características: Condição do item: NovoMarca: Bmw MotorradMPN: 11428409567Número de peça: 11428409567OEM: 11428409567Tempo de garantia: 6 mesesTipo de Garantia: Garantia de fábricaFILTRO DE OLEOVeículos (aplicações):Bmw Motorrad F 650 GS 2-Cyl 2007/2012Bmw Motorrad F 700 GS 2-Cyl 2012/2018Bmw Motorrad F 800 GS 2-Cyl 2007/2017Bmw Motorrad F 800 GS ADV 2-Cyl 2013/2017Bmw Motorrad F 800 GT 2-Cyl 2012/2019Bmw Motorrad F 800 R 2-Cyl 2009/2019Bmw Motorrad F 800 S 2-Cyl 2006/2010Bmw Motorrad F 800 ST 2-Cyl 2006/2012"
output = generate_output(input_text)
print(output)


1728409567 Filtro De Oleo Bmw Motorrad F 800 St 2-cyl 06-12 Marca: BMW Motorrad, Número de peça: 11428409567, OEM: 11428409567, DescriçãoFILTRO DE OLEO Bmw Motorrad F 800 ST 2-Cyl 06/12Características: Condição do item: NovoMarca: Bmw MotorradMPN: 11428409567Número de peça: 11428409567OEM: 11428409567Tempo de garantia: 6 mesesTipo de Garantia: Garantia de fábricaFILTRO DE OLEOVeículos (aplicações):Bmw Motorrad F 650 GS 2-Cyl 2007/2012Bmw Motorrad F 700 GS 2-Cyl 2012/2018Bmw Motorrad F 800 GS 2-Cyl 2007/2017Bmw Motorrad F 800 GS ADV 2-Cyl 2013/2017Bmw Motorrad F 800 GT 2-Cyl 2012/2019Bmw Motorrad F 800 R 2-Cyl 2009/2019Bmw Motorrad F 800 S 2-Cyl 2006/2010Bmw Motorrad F 800 ST 2-Cyl 2006/2012
{'catalog_catalog.code': '1728409567 Filtro De Oleo Bmw Motorrad F 800 St 2-cyl 06-12 Marca: BMW Motorrad', 'catalog_catalog.full_description': 'N/D', 'catalog_catalog.description': 'Número de peça: 11428409567', 'catalog_catalog.similar_parts': 'N/D', 'catalog_catalog.application_table': 'OEM: 1142